# Incorporating Other Software into Python Notebooks, e.g. ImageJ

**Part of the IAFIG-RMS *Python for Bioimage Analysis* Course.**

*Dr Chas Nelson* (based on https://github.com/imagej/tutorials)

2019-12-11 1000--1150

## Aim

To introduce the considerations requireed for incorporating other image analysis software pipelines into a Python notebook environment.

## ILOs

* Appreciate the challenges of incorporating other software into Python pipelines.
* Understand that ImageJ API concept.
* Recognise ImageJ API calls in the Java-native Groovy environment.

## Why Incorporate Other Software

* You may with wish to incorporate existing algorithsm or tools that you already use.
* You may wish to compare your new Python implementation with an existing tool (to show yours i better).
* You might have a colleage that refuses to use anything but FIJI.

## Challenges of Working With Other Software

* Many tools have APIs - infrastructures that allow you to programmatically access features.
* However, these could be experimental, poorly documented and/or unsupported.
* Worse, if the software your trying to use is written in a substantially different language, e.g. ImageJ and Java, you may need to find (or write) Python-specific wrappers for the Java-friendly API.

## Working with Experimental Codes

* Over the next few hours we will explore how ImageJ operationg an plugins can be called from within a Jupyter Notebook.
* This requires us to use layers and layers of experimental, in-development codes.
* There's a good chance something might not work.
  * And that we won't be able to work out why.
* However, we think being exposed to these tools is an important aspect to teach as these packages are being developed and may become powerful tools in the future.

## Loading an ImageJ Instance

* To access the ImageJ API we must first create an ImageJ instance (also called a gateway).
* This is essentially a hidden ImageJ that we can programatically control.
* The ImageJ instance gives us access to ImageJ Ops and Plugins.

In [ ]:
// Initialiase a specific version of ImageJ with FIJI plug-ins
// This is reproducible for all notebook users (but doesn't have everything)
 // %classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
// %classpath add mvn sc.fiji fiji 2.0.0-pre-10
// %classpath add mvn net.imagej imagej 2.0.0-rc-71

// Initialiase a local version of ImageJ with FIJI plug-ins
// This is reproducible for each user but not necesarily across computers (but has extras)
%classpath add jar '/home/chas/Fiji.app/jars/*'
%classpath add jar '/home/chas/Fiji.app/jars/bio-formats/*'
%classpath add jar '/home/chas/Fiji.app/plugins/*'

// Create an ImageJ gateway.
ij = new net.imagej.ImageJ()
"ImageJ v${ij.getVersion()} is ready to go."

## ImageJ Services

* The ImageJ instance provide a series of top-levels called 'services'.
* Each service provides a group of related tools.
* We'll only really use:
 * `.notebook()`, which provides a set of notebook-related tools,
 * `.io()`, which provides input/output tools,
 * `.op()`, which provides access to ImageJ Ops
 * and `.plugin()`, which provides access to plugins.

In [ ]:
// Get list of all services
ij.notebook().methods(ij).findAll{ it.get("returns").endsWith("Service") }

In [ ]:
// Identify methods available for an object, here a list
myList = ["quick", "brown", "fox"]
ij.notebook().methods(myList)

## Loading and Saving Data

### Loading data

In [ ]:
// Load a remote test image
myImageJImage = ij.io().open("https://imagej.nih.gov/ij/images/blobs.gif")

### Saving data

Similarly, you can use the I/O service to save data to an external destination:

In [ ]:
destPath = "./assets/blobs.png"
ij.io().save(myImageJImage, destPath)

## Plugins, Modules and Scripts

* Officially everything in ImageJ is a 'plugin' as it's built upon the SciJava plugin framework.
* In reality we can focus on two main types of plugins:
  * Module/commands
  * And ImageJ scripts

In [ ]:
// Get total number of plugins
pluginCount = ij.plugin().getIndex().size()
println("There are " + pluginCount + " plugins available.")

In [ ]:
// Get complete list of 'plugins'
kindCounts = [:]
ij.plugin().getPlugins().forEach{plugin ->
  kind = plugin.getPluginType().getName()
  // Comment the next two lines to show all standard ImageJ Ops
  if (kind.startsWith('net.imagej.ops.Ops') || kind.startsWith('net.imagej.ops.features'))
    kind = 'net.imagej.ops.Op'
  kindCounts.put(kind, kindCounts.getOrDefault(kind, 0) + 1)
}
// Display number of plugins for each class
kindCounts.keySet().sort().stream().map{kind -> [
    "Name": (kind =~ /\.[^\.]*$/)[0][1..-1],
    "Number": kindCounts.get(kind)
]}.collect()

* Plugins provide modules that can be called directly from the ImageJ API and usually match a standard ImageJ operation.
* Modules written in Java are usually called commands.
  * Java commands have advantages in performance, type safety and more.
* Modules written in other SciJava languages, e.g. Groovy, are usually called script.
  * Scripts have the benefit of being written in the language you already know.

In [ ]:
// Define a simple module using Java, i.e. a command
// This module takes an input and provides two outputs

import org.scijava.ItemIO;
import org.scijava.command.Command;
import org.scijava.plugin.Parameter;
import org.scijava.plugin.Plugin;

@Plugin(type = Command.class)
public class Hello implements Command {
  
  @Parameter
  private String name;
  
  @Parameter(type = ItemIO.OUTPUT)
  private String greeting;
  
  @Parameter(type = ItemIO.OUTPUT)
  private int length;
  
  @Override
  public void run() {
    greeting = "Hello, " + name + "!";
    length = name.length();
  }
}

// Save a reference to the class, for use in the next cell.
greetingCommand = Hello.class

In [ ]:
// Run the command
inputs = ["name": "John Jacob Jingleheimer Schmidt"]
module = ij.command().run(greetingCommand, true, inputs).get()

// Extract outputs
["greeting" : module.getOutput("greeting"),
 "length"   : module.getOutput("length")]

In [ ]:
// Write the same module as a script (as a string constant)
// It can then be passed to the script service.
script = """
#@input String name
#@output String greeting
#@output int length
greeting = "Hello, " + name + "!"
length = name.length()
"""

// Run the script
// Note the similarities to commands (above)
inputs = ["name": "John Jacob Jingleheimer Schmidt"]
module = ij.script().run("greeting.groovy", script, true, inputs).get();

// Extract outputs
// Note the similarities to commands (above)
["greeting" : module.getOutput("greeting"),
 "length"   : module.getOutput("length")]

## Ops

* If you scroll back to the list of plugins, you'll notice over 800 Ops.
* Ops are the core set of ImageJ processing tools and part of the ImageJ modular framework.

In [ ]:
// Get total number of ops (categories)
opsCount = ij.op().ops().size()
println("There are " + opsCount + " ops categories available.")

In [ ]:
import net.imagej.ops.OpUtils
opsByNS = [:]
ij.op().ops().each{op ->
  ns = OpUtils.getNamespace(op)
  name = OpUtils.stripNamespace(op)
  if (!opsByNS.containsKey(ns)) opsByNS.put(ns, name)
  else opsByNS.put(ns, opsByNS.get(ns) + ', ' + name)
}
opsByNS.put('<global>', opsByNS.remove(null))
display(opsByNS)

In [ ]:
// Find available operations with 'gauss' in the name
ij.op().help('gauss')

In [ ]:
// Smudge it up horizontally.
double[] horizSigmas = [8, 0, 0]
myImageJImageHBlur = ij.op().filter().gauss(myImageJImage, horizSigmas)

display(myImageJImage)
display(myImageJImageHBlur)

In [ ]:
display([
    "geometricMean": ij.op().stats().geometricMean(myImageJImage).toString(),
    "harmonicMean": ij.op().stats().harmonicMean(myImageJImage).toString(),
    "kurtosis": ij.op().stats().kurtosis(myImageJImage).toString(),
    "max": ij.op().stats().max(myImageJImage).toString(),
    "mean": ij.op().stats().mean(myImageJImage).toString(),
    "median": ij.op().stats().median(myImageJImage).toString(),
    "min": ij.op().stats().min(myImageJImage).toString(),
    "moment1AboutMean": ij.op().stats().moment1AboutMean(myImageJImage).toString(),
    "moment2AboutMean": ij.op().stats().moment2AboutMean(myImageJImage).toString(),
    "moment3AboutMean": ij.op().stats().moment3AboutMean(myImageJImage).toString(),
    "moment4AboutMean": ij.op().stats().moment4AboutMean(myImageJImage).toString(),
    "size": ij.op().stats().size(myImageJImage).toString(),
    "skewness": ij.op().stats().skewness(myImageJImage).toString(),
    "stdDev": ij.op().stats().stdDev(myImageJImage).toString(),
    "sum": ij.op().stats().sum(myImageJImage).toString(),
    "sumOfInverses": ij.op().stats().sumOfInverses(myImageJImage).toString(),
    "sumOfLogs": ij.op().stats().sumOfLogs(myImageJImage).toString(),
    "sumOfSquares": ij.op().stats().sumOfSquares(myImageJImage).toString(),
    "variance": ij.op().stats().variance(myImageJImage).toString()
])

## Summary

* Incorporating other software into Python pipelines has both benefits and challenges.
* The ImageJ API and ImageJ instance allow programmatic access to ImageJ commands and scripts.
* ImageJ Ops provide the bulk of ImageJ features